# Evaluation

In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('../../')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import datetime

import numpy as np
import pandas as pd

from src.data import train_test_split, MRISequence
from src.model import create_model, compile_model, load_checkpoint
from src.model.evaluation import show_metrics

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")

plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['image.cmap'] = 'viridis'

%config InlineBackend.figure_format='retina'
plt.rcParams.update({'font.size': 15})

In [3]:
from src.evaluation.segmentation_masks import SegmentationMasksSaver

ROOT_DIR = '../../tmp'

saver = SegmentationMasksSaver(ROOT_DIR, fname="evaluation--76.csv")

In [4]:
# saver.df.head()

In [5]:
# list(saver.df.columns)

In [6]:
df = saver.df

def ins_del_fn(ins_col, del_col):
    def fn(row):
        value = (row[ins_col] + (1 - row[del_col])) / 2
        return value
    return fn

for cls in ['AD+CN', 'AD', 'CN']:
# for cls in ['AD+CN']:
    df[f'ins_del__auc_median__{cls}'] = df.apply(ins_del_fn(f'insertion__auc_median__{cls}', f'deletion__auc_median__{cls}'), axis=1)

In [7]:
columns = ['notebook_key']

# for cls in ['AD+CN']: 
for cls in ['AD+CN', 'AD', 'CN']: 
    for m in [
        'ins_del__auc_median__',
#         'arr_heat_sum_density__1__50%__',
#         'arr_heat_sum_density__2__50%__',
#         'arr_heat_sum_density__3__50%__',
#         'arr_heat_sum_density__4__50%__',
        'arr_heat_sum_non_brain_vs_brain__50%__',
        'arr_heat_sum_0_vs_1_3_4__50%__',
        'arr_heat_sum_0_2_vs_1_3_4__50%__',
    ]:
        columns.append(f'{m}{cls}')
        
for cls in ['AD+CN']: 
# for cls in ['AD+CN', 'AD', 'CN']: 
    for m in [
#         'ins_del__auc_median__',
        'arr_heat_sum_density__0__50%__',
        'arr_heat_sum_density__1__50%__',
        'arr_heat_sum_density__2__50%__',
        'arr_heat_sum_density__3__50%__',
        'arr_heat_sum_density__4__50%__',
#         'arr_heat_sum_non_brain_vs_brain__50%__',
#         'arr_heat_sum_0_vs_1_3_4__50%__',
#         'arr_heat_sum_0_2_vs_1_3_4__50%__',
    ]:
        columns.append(f'{m}{cls}')
        
new_names = {}

for col in columns:
    new_name = col.replace('50%', '')
    new_name = new_name.replace('median', '')
    new_name = new_name.replace('AD', '(AD)')
    new_name = new_name.replace('CN', '(CN)')
    new_name = new_name.replace('(AD)+(CN)', '(AD + CN)')
    new_name = new_name.replace('_', ' ')
    new_name = new_name.replace('   ', ' ')
    new_name = new_name.replace('  ', ' ')
    new_name = new_name.replace('arr heat sum density 0', 'B - nie mozog')
    new_name = new_name.replace('arr heat sum density 1', 'B - šedá hmota')
    new_name = new_name.replace('arr heat sum density 2', 'B - biela hmota')
    new_name = new_name.replace('arr heat sum density 3', 'B - komory')
    new_name = new_name.replace('arr heat sum density 4', 'B - hipokampus')
    new_name = new_name.replace('ins del auc', 'A')
    new_name = new_name.replace('arr heat sum non brain vs brain', 'C')
    new_name = new_name.replace('arr heat sum 0 vs 1 3 4', 'E')
    new_name = new_name.replace('arr heat sum 0 2 vs 1 3 4', 'D')
    new_names[col] = new_name
        
def map_notebook_key(row):
    if 'captum--guided-backprop' in row['notebook_key']:
        row['notebook_key'] = 'Guided Backprop'
    if 'captum--guided-grad-cam' in row['notebook_key']:
        row['notebook_key'] = 'Guided GradCAM'
    if 'captum--grad-cam' in row['notebook_key']:
        row['notebook_key'] = 'GradCAM'
    if 'b2-value+1' in row['notebook_key']:
        row['notebook_key'] = 'RISE (b2_value = 1)'
    if 'b1+1' in row['notebook_key']:
        row['notebook_key'] = 'RISEI'
    if 'b2-value+0' in row['notebook_key']:
        row['notebook_key'] = 'RISE (b2_value = 0)'
    if 'guided' in row['notebook_key']:
        row['notebook_key'] = 'Guided RISE (b2_value = 0)'
    return row    
    
q_df = df[columns]

def fix_floats(row):
    for col in list(q_df.columns):
        print(col)
        if col != 'notebook_key':
            row[col] = '%0.3f' % row[col]
    return row

q_df = q_df.apply(fix_floats, axis=1)
q_df = q_df.apply(map_notebook_key, axis=1)
q_df = q_df.rename(columns=new_names)
new_columns = list(q_df.columns)
new_columns.sort()
new_columns.remove('notebook key')
new_columns.insert(0, 'notebook key')

q_df = q_df[new_columns]
q_df = q_df.sort_values(by='notebook key')
q_df = q_df.rename(columns={'notebook key': 'Metóda'})
q_df

notebook_key
ins_del__auc_median__AD+CN
arr_heat_sum_non_brain_vs_brain__50%__AD+CN
arr_heat_sum_0_vs_1_3_4__50%__AD+CN
arr_heat_sum_0_2_vs_1_3_4__50%__AD+CN
ins_del__auc_median__AD
arr_heat_sum_non_brain_vs_brain__50%__AD
arr_heat_sum_0_vs_1_3_4__50%__AD
arr_heat_sum_0_2_vs_1_3_4__50%__AD
ins_del__auc_median__CN
arr_heat_sum_non_brain_vs_brain__50%__CN
arr_heat_sum_0_vs_1_3_4__50%__CN
arr_heat_sum_0_2_vs_1_3_4__50%__CN
arr_heat_sum_density__0__50%__AD+CN
arr_heat_sum_density__1__50%__AD+CN
arr_heat_sum_density__2__50%__AD+CN
arr_heat_sum_density__3__50%__AD+CN
arr_heat_sum_density__4__50%__AD+CN
notebook_key
ins_del__auc_median__AD+CN
arr_heat_sum_non_brain_vs_brain__50%__AD+CN
arr_heat_sum_0_vs_1_3_4__50%__AD+CN
arr_heat_sum_0_2_vs_1_3_4__50%__AD+CN
ins_del__auc_median__AD
arr_heat_sum_non_brain_vs_brain__50%__AD
arr_heat_sum_0_vs_1_3_4__50%__AD
arr_heat_sum_0_2_vs_1_3_4__50%__AD
ins_del__auc_median__CN
arr_heat_sum_non_brain_vs_brain__50%__CN
arr_heat_sum_0_vs_1_3_4__50%__CN
arr_hea

,Metóda,A (AD + CN),A (AD),A (CN),B - biela hmota (AD + CN),B - hipokampus (AD + CN),B - komory (AD + CN),B - nie mozog (AD + CN),B - šedá hmota (AD + CN),C (AD + CN),C (AD),C (CN),D (AD + CN),D (AD),D (CN),E (AD + CN),E (AD),E (CN)
2,GradCAM,0.556,0.593,0.531,0.458,0.407,0.428,0.501,0.445,1.140,0.852,1.221,1.113,0.951,1.163,1.137,0.906,1.203
0,Guided Backprop,0.682,0.786,0.574,0.607,0.555,0.330,0.464,0.570,0.808,0.808,0.809,0.875,0.878,0.872,0.836,0.837,0.834
1,Guided GradCAM,0.679,0.788,0.546,0.508,0.501,0.489,0.497,0.505,0.983,0.942,0.999,0.990,0.971,0.999,0.986,0.957,0.998
4,RISE (b2_value = 0),0.395,0.367,0.404,0.492,0.487,0.499,0.501,0.497,1.010,0.998,1.016,1.002,0.999,1.006,1.008,0.980,1.017
3,RISE (b2_value = 1),0.618,0.540,0.640,0.500,0.449,0.528,0.498,0.501,0.992,1.024,0.986,0.991,0.991,0.989,0.996,1.003,0.987
5,RISEI,0.479,0.498,0.460,0.505,0.513,0.490,0.499,0.499,0.994,1.028,0.988,0.999,0.992,1.006,1.005,1.009,1.004


In [16]:
q_df.to_csv('evaluation--78.csv', index=False)

In [9]:
new_names

{'notebook_key': 'notebook key',
 'ins_del__auc_median__AD+CN': 'A (AD + CN)',
 'arr_heat_sum_non_brain_vs_brain__50%__AD+CN': 'C (AD + CN)',
 'arr_heat_sum_0_vs_1_3_4__50%__AD+CN': 'E (AD + CN)',
 'arr_heat_sum_0_2_vs_1_3_4__50%__AD+CN': 'D (AD + CN)',
 'ins_del__auc_median__AD': 'A (AD)',
 'arr_heat_sum_non_brain_vs_brain__50%__AD': 'C (AD)',
 'arr_heat_sum_0_vs_1_3_4__50%__AD': 'E (AD)',
 'arr_heat_sum_0_2_vs_1_3_4__50%__AD': 'D (AD)',
 'ins_del__auc_median__CN': 'A (CN)',
 'arr_heat_sum_non_brain_vs_brain__50%__CN': 'C (CN)',
 'arr_heat_sum_0_vs_1_3_4__50%__CN': 'E (CN)',
 'arr_heat_sum_0_2_vs_1_3_4__50%__CN': 'D (CN)',
 'arr_heat_sum_density__0__50%__AD+CN': 'B - nie mozog (AD + CN)',
 'arr_heat_sum_density__1__50%__AD+CN': 'B - šedá hmota (AD + CN)',
 'arr_heat_sum_density__2__50%__AD+CN': 'B - biela hmota (AD + CN)',
 'arr_heat_sum_density__3__50%__AD+CN': 'B - komory (AD + CN)',
 'arr_heat_sum_density__4__50%__AD+CN': 'B - hipokampus (AD + CN)'}

In [10]:
q_df

,Metóda,A (AD + CN),A (AD),A (CN),B - biela hmota (AD + CN),B - hipokampus (AD + CN),B - komory (AD + CN),B - nie mozog (AD + CN),B - šedá hmota (AD + CN),C (AD + CN),C (AD),C (CN),D (AD + CN),D (AD),D (CN),E (AD + CN),E (AD),E (CN)
2,GradCAM,0.556,0.593,0.531,0.458,0.407,0.428,0.501,0.445,1.140,0.852,1.221,1.113,0.951,1.163,1.137,0.906,1.203
0,Guided Backprop,0.682,0.786,0.574,0.607,0.555,0.330,0.464,0.570,0.808,0.808,0.809,0.875,0.878,0.872,0.836,0.837,0.834
1,Guided GradCAM,0.679,0.788,0.546,0.508,0.501,0.489,0.497,0.505,0.983,0.942,0.999,0.990,0.971,0.999,0.986,0.957,0.998
4,RISE (b2_value = 0),0.395,0.367,0.404,0.492,0.487,0.499,0.501,0.497,1.010,0.998,1.016,1.002,0.999,1.006,1.008,0.980,1.017
3,RISE (b2_value = 1),0.618,0.540,0.640,0.500,0.449,0.528,0.498,0.501,0.992,1.024,0.986,0.991,0.991,0.989,0.996,1.003,0.987
5,RISEI,0.479,0.498,0.460,0.505,0.513,0.490,0.499,0.499,0.994,1.028,0.988,0.999,0.992,1.006,1.005,1.009,1.004


In [11]:
new_data = {'notebook_key': [], 'metric': [], 'value': []}
df = saver.df.set_index('notebook_key')
for notebook_key, values in df.iterrows():
    for key, value in values.items():
        new_data['notebook_key'].append(notebook_key)
        new_data['metric'].append(key)
        new_data['value'].append(value)

new_df = pd.DataFrame(data=new_data)

In [12]:
new_df.head()

,notebook_key,metric,value
0,captum--guided-backprop--76,y_true__mean__AD+CN,0.500000
1,captum--guided-backprop--76,y_pred__mean__AD+CN,0.497221
2,captum--guided-backprop--76,arr_heat_sum__0__mean__AD+CN,432171.589638
3,captum--guided-backprop--76,arr_heat_sum__1__mean__AD+CN,151296.612870
4,captum--guided-backprop--76,arr_heat_sum__2__mean__AD+CN,102348.021382


In [13]:
# higher is better
new_df[new_df['metric'] == 'insertion__auc_median']

,notebook_key,metric,value


In [14]:
# lower is better
new_df[new_df['metric'] == 'deletion__auc_median']

,notebook_key,metric,value


In [15]:
# gain brain vs non-brain areas, higher is better
new_df[new_df['metric'] == 'arr_heat_sum_gain_other__50%']

,notebook_key,metric,value
